In [ ]:
#Continously gets real time stock data and updates every 10 minutes through the day

In [192]:
# IMPORTS #
import time
t0 = time.clock()

import pandas as pd
from pandas.tseries.offsets import BDay
import numpy as np
import datetime as dt
import sched
from copy import copy
import warnings

warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)


In [182]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds', 
            'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
client = gspread.authorize(creds)

# Find a workbook by name and open the first sheet
# Make sure you use the right name here.

stockList = client.open("STOCKS LIST").sheet1 #stock list

#real time sheets
APPL_rTime = client.open("AAPL_rTime").sheet1
TSLA_rTime = client.open("TSLA_rTime").sheet1
MSFT_rTime = client.open("MSFT_rTime").sheet1
VZ_rTime = client.open("VZ_rTime").sheet1
AMZN_rTime = client.open("AMZN_rTime").sheet1

#legacy sheets
AAPL_data = client.open("AAPL_data").sheet1  #legacy
VZ_data = client.open("VZ_data").sheet1 #legacy
AMZN_data = client.open("AMZN_data").sheet1 #legacy
MSFT_data = client.open("MSFT_data").sheet1 #legacy
TSLA_data = client.open("TSLA_data").sheet1 #legacy

# Extract and print all of the values
list_of_hashes = stockList.get_all_records()
print(list_of_hashes)
#setUpSheetsClient()

[{'STOCKS': 'AAPL'}, {'STOCKS': 'MSFT'}, {'STOCKS': 'AMZN'}, {'STOCKS': 'VZ'}, {'STOCKS': 'TSLA'}]


In [183]:
# ================================================================== #
# filepath management

project_dir = r'/Users/jamesgalante/Documents/College/Semester 7/CPEG498/Uptick/Legacy Data/get past month data/' 
price_path = project_dir + r'Symbols/'
apikey = '0adbf00b462c1acca954a43d94279b92'


In [184]:
# ================================================================== #
# datetime management

d = dt.date.today()
# ---------- Days ---------- 
l10 = d - 10 * BDay()
l21 = d - 21 * BDay()
l63 = d - 63 * BDay()
l252 = d - 252 * BDay()
# ---------- Years ---------- 
l252_x2 = d - 252 * 2 * BDay() 
l252_x3 = d - 252 * 3 * BDay() 
l252_x5 = d - 252 * 5 * BDay()
l252_x7 = d - 252 * 7 * BDay() 
l252_x10 = d - 252 * 10 * BDay() 
l252_x20 = d - 252 * 20 * BDay() 
l252_x25 = d - 252 * 25 * BDay()


In [185]:
##points to our google sheet "STOCKS LIST" gets the values and makes it a pandas dataframe to manipulate
def getStockList():
    sheetsList = stockList.get_all_values()
    sheetsList = sheetsList[1:] #remove "STOCKS" title
    ##print(sheetsList)
    sheetsList = flatten(sheetsList)
    #print(sheetsList)
    #create new df 
    df = pd.DataFrame({'STOCKS':sheetsList})
    return df


In [186]:
##takes a 2d list and flattens it to 1d
def flatten(input):
    new_list = []
    for i in input:
        for j in i:
            new_list.append(j)
    return new_list

In [187]:
#setting list of stock symbols to var syms
syms = getStockList() #dataframe of the stocks we are getting data for
print(syms)

  STOCKS
0   AAPL
1   MSFT
2   AMZN
3     VZ
4   TSLA


In [188]:
#BARCHART API
def construct_barChart_url(sym, api_key=apikey):
    '''Function to construct barchart api url'''
    
    url = 'http://marketdata.websol.barchart.com/getQuote.csv?' +\
            'key={}&symbols='.format(api_key, sym)
    return url

In [189]:
import ondemand
# get quote data for Apple and Microsoft
def getStockLiveData(stock):
    od = ondemand.OnDemandClient(api_key=apikey, end_point='https://marketdata.websol.barchart.com/')
    quote = od.quote(stock)['results']
    print(quote)
    return (quote)

getStockLiveData('AAPL')

Barchart OnDemand Client: https://marketdata.websol.barchart.com/
[{'symbol': 'AAPL', 'exchange': 'BATS', 'name': 'Apple Inc', 'dayCode': 'M', 'serverTimestamp': '2018-10-23T14:36:32-05:00', 'mode': 'i', 'lastPrice': 222.79, 'tradeTimestamp': '2018-10-23T15:21:31-05:00', 'netChange': 2.14, 'percentChange': 0.97, 'unitCode': '2', 'open': 217.7, 'high': 223.01, 'low': 214.71, 'close': 0, 'flag': '', 'volume': 2009767}]


[{'close': 0,
  'dayCode': 'M',
  'exchange': 'BATS',
  'flag': '',
  'high': 223.01,
  'lastPrice': 222.79,
  'low': 214.71,
  'mode': 'i',
  'name': 'Apple Inc',
  'netChange': 2.14,
  'open': 217.7,
  'percentChange': 0.97,
  'serverTimestamp': '2018-10-23T14:36:32-05:00',
  'symbol': 'AAPL',
  'tradeTimestamp': '2018-10-23T15:21:31-05:00',
  'unitCode': '2',
  'volume': 2009767}]

In [190]:
def sendDataToSheets():
    stocks = syms['STOCKS'].values.tolist()
    for s in stocks:
        time.sleep(0.21)
        print(s)
        stockData = getStockLiveData(s)
        print(stockData)

        timestamp = str(dt.datetime.now().time())

        if(s == 'AAPL'):
            print("adding AAPL")
            APPL_rTime.append_row([timestamp, stockData[0]['symbol'], str(dt.date.today()), stockData[0]['lastPrice'], stockData[0]['open'], stockData[0]['high'], stockData[0]['low'], stockData[0]['netChange'], stockData[0]['volume']])    
        elif (s == 'VZ'):
            print("adding VZ")
            VZ_rTime.append_row([timestamp, stockData[0]['symbol'], str(dt.date.today()), stockData[0]['lastPrice'], stockData[0]['open'], stockData[0]['high'], stockData[0]['low'], stockData[0]['netChange'], stockData[0]['volume']])    
        elif (s == 'TSLA'):
            print("adding TSLA")
            TSLA_rTime.append_row([timestamp, stockData[0]['symbol'], str(dt.date.today()), stockData[0]['lastPrice'], stockData[0]['open'], stockData[0]['high'], stockData[0]['low'], stockData[0]['netChange'], stockData[0]['volume']])    
        elif (s == 'AMZN'):
            print("adding AMZN")
            AMZN_rTime.append_row([timestamp, stockData[0]['symbol'], str(dt.date.today()), stockData[0]['lastPrice'], stockData[0]['open'], stockData[0]['high'], stockData[0]['low'], stockData[0]['netChange'], stockData[0]['volume']])    
        elif (s == 'MSFT'):
            print("adding MSFT")
            MSFT_rTime.append_row([timestamp, stockData[0]['symbol'], str(dt.date.today()), stockData[0]['lastPrice'], stockData[0]['open'], stockData[0]['high'], stockData[0]['low'], stockData[0]['netChange'], stockData[0]['volume']])    


In [ ]:
s = sched.scheduler(time.time, time.sleep)
delay = 60 #seconds

def main(sc): 
    print("call again in 15 sec")
    sendDataToSheets()
    s.enter(delay, 1, main, (sc,))

s.enter(delay, 1, main, (s,))
s.run()

In [ ]:
now = datetime.now()
